In [ ]:
from lifelines import CoxPHFitter, datasets
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import util.cleaning_tools as tools


import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import RandomOverSampler, SMOTE

import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,\
precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score
from sklearn.feature_selection import SelectKBest, f_classif, chi2,f_regression
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression, LogisticRegression

from random import sample
import time
import warnings

In [ ]:
dataset =pd.read_csv(r"../tables/output/dataset", index_col=0)

def preprocess(df):
    df["censored_dtm"] = "2019-12-31"
    df["censored_duration_hour"] = (pd.to_datetime(df["censored_dtm"]) - pd.to_datetime(df["pre_dtm"])).astype(int) / (10**9*60*60)
    df["death_censored_duration_hour"] = df["diff_in_hour_death_date"] - df["pre_diff_hour"]
    df["sex"] = df["sex"].apply(lambda x : 0 if x=='M'else 1)
    df["duration"] = df["diab_diff_hour"] - df["pre_diff_hour"]
    df["duration"] = df["duration"].fillna(df["death_censored_duration_hour"])
    df["duration"] = df["duration"].fillna(df["censored_duration_hour"])
    df["flag"] = df["diab_age"].notnull().astype("int")

In [ ]:
demo_info =['pseudo_patient_key',
            'pre_age',
            'duration',
           'flag']
# select missingness less than 25% tets and HBA1C
tests_dict = {5200289: "cholesHDL",
 5200290: "choles",
 5200295: "creatinine",
 5200306: "fastingGlucose",
 5200325: "triglyceride"
}
valid_features = [
    'cholesHDL', 
    'creatinine', 
    'fastingGlucose', 
    'triglyceride',
    'HBA1C',
    'choles'
    ]
tests_name = list(tests_dict.values())

In [ ]:
# preprocess(dataset)
ds = dataset.copy()[demo_info + valid_features]
# ds = ds[["sex", "pre_age", "label", "duration"]+tests_name]
# ds = ds.dropna(how='any')

In [ ]:
ds

In [ ]:
ds = ds.dropna(how='any')

In [ ]:
# Imputation
lr = LogisticRegression()
numeric_col = tests_name + ["pre_age", "sex", "pre_diff_hour"]
# imp = IterativeImputer(estimator=lr, missing_values=np.nan,max_iter=10, verbose=2, imputation_order='roman', random_state=0)
imp = IterativeImputer(missing_values=np.nan, max_iter=10, verbose=2, imputation_order='roman', random_state=0)
ds[numeric_col] = imp.fit_transform(ds[numeric_col])
ds[numeric_col] = ds[numeric_col].clip(0)

In [ ]:
ds

In [ ]:
cph = CoxPHFitter(penalizer=0.1)
cox_train = ds[["pre_age", "flag", "duration"] + valid_features]
cph.fit(cox_train, duration_col='duration', event_col='flag')
cph.print_summary()

In [ ]:
cox_train.groupby("label")["pre_age"].mean()

In [ ]:
cph.check_assumptions(cox_train)